In [1]:
import pandas as pd
import plotly.graph_objects as go
from scipy.signal import argrelextrema
import numpy as np


In [2]:
df = pd.read_csv("EURUSD_Candlestick_1_D_ASK_05.05.2003-19.10.2019.csv")
df.tail()

,Local time,open,high,low,close,volume
4322,14.10.2019 00:00:00.000 GMT+0300,1.10399,1.10428,1.10129,1.10281,150329.9600
4323,15.10.2019 00:00:00.000 GMT+0300,1.10281,1.10463,1.09915,1.10338,225114.4925
4324,16.10.2019 00:00:00.000 GMT+0300,1.10331,1.10857,1.10226,1.10720,224179.4392
4325,17.10.2019 00:00:00.000 GMT+0300,1.10724,1.11400,1.10653,1.11254,221922.6500
4326,18.10.2019 00:00:00.000 GMT+0300,1.11256,1.11727,1.11149,1.11723,143073.7750


In [3]:

dfs = df.iloc[:500].copy()
dfs.tail()

,Local time,open,high,low,close,volume
495,23.03.2005 00:00:00.000 GMT+0200,1.30847,1.30856,1.29570,1.29855,1108449.303
496,24.03.2005 00:00:00.000 GMT+0200,1.29851,1.30310,1.29299,1.29366,1107288.303
497,25.03.2005 00:00:00.000 GMT+0200,1.29368,1.29687,1.29249,1.29562,1097519.010
498,28.03.2005 01:00:00.000 GMT+0300,1.29601,1.29632,1.28554,1.28962,1125835.914
499,29.03.2005 01:00:00.000 GMT+0300,1.28927,1.29551,1.28745,1.29216,1126423.191


In [4]:



def resistance(dfs,order):
    high = dfs.high.values
    
    maxhigh_rowdx = list(argrelextrema(high, np.greater, order=order)[0])
    minhigh_rowdx = list(argrelextrema(high, np.less, order=order)[0])
    rowdx = maxhigh_rowdx + minhigh_rowdx
    resistance = high[rowdx]

    
    return rowdx,resistance
    

    
def support(dfs,order):
    low = dfs.low.values
       
    maxlow_rowdx = list(argrelextrema(low, np.greater, order=order)[0])
    minlow_rowdx = list(argrelextrema(low, np.less, order=order)[0])
    rowdx = maxlow_rowdx + minlow_rowdx
    rowdx.sort()
    support = low[rowdx]
    
    return rowdx,support



In [5]:
patterndetected= []
def pattern(dfs):
    length = len(dfs)
    signal1 = length * [0]
    

    

    for row in range(1, length):

        max_rowdx = list(argrelextrema(dfs.close.values[:row], np.greater, order=10)[0])
        min_rowdx = list(argrelextrema(dfs.close.values[:row], np.less, order=10)[0])

        rowdx = max_rowdx + min_rowdx + [len(dfs.close.values[:row]) - 1]

        rowdx.sort()

        if len(rowdx) >= 5:

            current_rowdx = rowdx[-5:] # gets 5th from last to last extrema x values as such in idx
            

            

            current_pat = dfs.close.values[current_rowdx] # the corresponding prices values of the extrema

            XA = current_pat[1] - current_pat[0]
            AB = current_pat[2] - current_pat[1]
            BC = current_pat[3] - current_pat[2]
            CD = current_pat[4] - current_pat[3]

            err = 0.5
            ABr = np.array([0.618 - err, 0.618 + err]) * abs(XA)
            BCr = np.array([0.382 - err, 0.886 + err]) * abs(AB)
            CDr = np.array([1.27 - err, 1.618 + err]) * abs(BC)

            ### Bullisih
            if XA > 0 and AB < 0 and BC > 0 and CD < 0:
            
                

                if (ABr[0] < abs(AB) < ABr[1]
                 and BCr[0] < abs(BC) < BCr[1] 
                 and CDr[0] < abs(CD) < CDr[1]):


                    patterndetected.append((current_rowdx,current_pat,'sell'))
                    signal1[current_rowdx[4]] = 1
                    
            
	        
            elif XA<0 and AB>0 and BC<0 and CD>0:
                if (ABr[0]<abs(AB)<ABr[1] 
			    and BCr[0]<abs(BC)<BCr[1] 
			    and CDr[0]<abs(CD)<CDr[1]):
                    patterndetected.append((current_rowdx,current_pat,'buy'))
                    signal1[current_rowdx[4]] = 2

            else:
                signal1[current_rowdx[4]] = 0

    return signal1
    


In [6]:
dfs['signal1'] = pattern(dfs) 

dfs[dfs['signal1']==0].count()

Local time    488
open          488
high          488
low           488
close         488
volume        488
signal1       488
dtype: int64

In [7]:



rr,aa =resistance(dfs,10)
sindex,sprice =support(dfs,10)








In [8]:


fig = go.Figure(data=[go.Candlestick(x=dfs.index,
                open=dfs['open'],
                high=dfs['high'],
                low=dfs['low'],
                close=dfs['close'],
                line=dict(width=1), opacity=1,
                increasing_fillcolor='#24A06B',
                decreasing_fillcolor="#CC2E3C",
                increasing_line_color='#2EC886',  
                decreasing_line_color='#FF3A4C'
                )])

for a,b in zip(rr,aa):
    fig.add_trace(go.Scatter(x=[a,a+7], y=[b,b],
                        mode='lines',
                        name=f'resiatance '))

for a,b in zip(sindex,sprice):
    fig.add_trace(go.Scatter(x=[a,a+7], y=[b,b],
                        mode='lines',
                        name=f'support '))


for i in patterndetected:
    fig.add_trace(go.Scatter(x=i[0], y=i[1],
                        mode='lines+markers',
                        name=f'Harmonic {i[2]}'))

fig.show()                